### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [1]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [2]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [3]:
#Function that creates lump volumes
def LumpVolumes(PDoperator, gridgenerator):
    
    totnode_org = PDoperator.totnode_org 
    totnode = PDoperator.totnode 
    xvec_org = PDoperator.xvec_org
    
    xvec = gridgenerator.xvec 
    dvolume = gridgenerator.dvolume
    
    
    dvolumes_org = [0] * totnode_org
    tiny = 1E-7
    rho = [0]*totnode_org
    dvolume_org = [0]*totnode_org
    for i in range(totnode):
        rhosum = 0.0
        for j in range(totnode_org):
            rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
            rhosum = rhosum + rho[j]
        for j in range(totnode_org):
            dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
    rhosum = 0.0;
    for j in range(totnode_org):
        rhosum += dvolume_org[j]
    print("Total volume of output points         = ",totnode*dvolume[0] )
    print("Total volume of computed input points = ", rhosum)
    return dvolume_org

In [4]:
#Calculates size of Peridynamic operator
def getSize1D(n1order):
    nsize = n1order + 1
    return nsize

In [5]:
#Function that sets the order of the Differential Operators
def SetDiffOperators1D(nsize):
    order = list(range(0,nsize))
    return order

In [6]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
def GenerateNodeFamily(xvec_org, xvec):
    xsi_order = pd.DataFrame({'node':xvec_org})
    xsivec = pd.DataFrame({'node':xvec_org})
    #xsivec = pd.DataFrame()
    
    #Ordering nodes
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(math.sqrt((xvec[i]-xvec_org[j])**2))
        xsi_order[str(xvec[i])] = aux
        xsivec[str(xvec[i])] = aux
    
    xsi_order.set_index('node', inplace=True)
    for i in xsi_order:
        xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
    xsi_order_2 = pd.DataFrame({'node':xvec_org})
    for i in xsi_order:
        xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
    #Calculating deltas (horizon) we are selecting the largest bond
    deltax = []
    for i in xsivec:
        deltax.append(max(list(xsivec[str(i)])))
    return deltax;

In [7]:
import numpy as np
import math
#Function that calculates the 1D pass derivative operator
def Derivative_Operator_1D_PASS(PDoperator, gridgenerator):
    
    xvec = gridgenerator.xvec
    n1order = PDoperator.n1order
    PDoperator.nsize = getSize1D(n1order)
    PDoperator.order = SetDiffOperators1D(PDoperator.nsize)
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    DiffAmatInv1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmatInv 2D matrix
    DiffBvec1D = [0] * PDoperator.nsize
    DiffAvec1D = [0] * PDoperator.nsize
    plist = [0] * PDoperator.nsize
    blist = [0] * PDoperator.nsize
    weight = [0] * PDoperator.nsize
    rcvec = [0] * PDoperator.nsize
    wgt = [0] * PDoperator.totnode_org
    ndfam = [0] * PDoperator.totnode_org
    
   
    nsize = PDoperator.nsize
    
    xvec_org = PDoperator.xvec_org
    #Here is where we should calculate A matrix, inverse of A matrix and B vector; for every Peridynamic node.
    
    
    #Generate node family
    g = GenerateNodeFamily(xvec_org, xvec)
    
    #Form A matrix
        
    #Form B vector
        
    #print("Interpolating for N=2")
    #for i in range(PDoperator.totnode):
        #GenerateNodeFamily_0
        
    
   
    #print(PDoperator.order)
    return g
    
    

In [8]:
#The PDoperator class constructor has the following inputs
#PDoperator_name = PDoperator(number_input_points, xmin, xmax, output_points, 
#                             xvec_org, yvec_org)
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)
print(xvec_org)
#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator, gridgenerator)
#Calculating the 1D derivative operator
h = Derivative_Operator_1D_PASS(PDoperator, gridgenerator)

#for i in h:
#        print(list(h[str(i)].sort_values()))

#print(h['8.9775'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Total volume of output points         =  9.0
Total volume of computed input points =  9.0


In [9]:
print(h)
#h[str(i)].sort_values().keys()
#h_0 = 

[9, 8.9775, 8.9325, 8.8875, 8.8425, 8.7975, 8.7525, 8.7075, 8.6625, 8.6175, 8.5725, 8.5275, 8.4825, 8.4375, 8.3925, 8.3475, 8.3025, 8.2575, 8.2125, 8.1675, 8.1225, 8.0775, 8.0325, 7.9875, 7.9425, 7.8975, 7.8525, 7.8075, 7.7625, 7.7175, 7.6725, 7.6274999999999995, 7.5825, 7.5375, 7.4925, 7.4475, 7.4025, 7.3575, 7.3125, 7.2675, 7.2225, 7.1775, 7.1325, 7.0875, 7.0425, 6.9975000000000005, 6.952500000000001, 6.907500000000001, 6.8625, 6.8175, 6.7725, 6.7275, 6.6825, 6.6375, 6.592499999999999, 6.547499999999999, 6.5024999999999995, 6.4575, 6.4125, 6.3675, 6.3225, 6.2775, 6.2325, 6.1875, 6.1425, 6.0975, 6.0525, 6.0075, 5.9625, 5.9175, 5.8725000000000005, 5.827500000000001, 5.782500000000001, 5.7375, 5.6925, 5.6475, 5.6025, 5.5575, 5.5125, 5.467499999999999, 5.422499999999999, 5.3774999999999995, 5.3325, 5.2875, 5.2425, 5.1975, 5.1525, 5.1075, 5.0625, 5.0175, 4.9725, 4.9275, 4.8825, 4.8375, 4.7925, 4.7475, 4.7025, 4.6575, 4.6125, 4.5675, 4.5225, 4.5225, 4.5675, 4.6125, 4.6575, 4.7025, 4.7475, 

In [11]:
#h.to_excel("output.xlsx")

In [ ]:
print(gridgenerator.xvec)
print(xvec_org[2]-gridgenerator.xvec[0])

In [ ]:
print('Hello World')